In [2]:
import os

In [3]:
%pwd

'c:\\Users\\vishw\\Documents\\cap_project_tulasi\\YOLO-missile-detection-capstone-project\\research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\vishw\\Documents\\cap_project_tulasi\\YOLO-missile-detection-capstone-project'

In [6]:
from dataclasses import dataclass

@dataclass
class ModelPredictionConfig:
  trained_weights_path: str
  test_images_path: str
  results_path: str
  confidence_threshold: float 


In [7]:
from dataclasses import dataclass
from YOLOv8_objectdetection.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from YOLOv8_objectdetection.utils.common import read_yaml, create_directories
from YOLOv8_objectdetection import logger

In [8]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)

    def get_model_prediction_config(self):
        # Assuming your params.yaml structure and keys, with added handling for missing keys
        confidence_threshold = self.params.get('prediction_params', {}).get('confidence_threshold', 0.25)  # Provide default value
        
        # Assuming 'model_prediction' is correctly loaded from your config
        config = self.config['model_prediction']

        return ModelPredictionConfig(
            trained_weights_path=config.get('trained_weights_path'),
            test_images_path=config.get('test_images_path'),
            results_path=config.get('results_path'),
            confidence_threshold=confidence_threshold,
        )


In [9]:
from dataclasses import dataclass
from YOLOv8_objectdetection.constants.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from YOLOv8_objectdetection.utils.common import read_yaml, create_directories
from YOLOv8_objectdetection import logger
from pathlib import Path
from ultralytics import YOLO
import os

class YOLOInference:
    def __init__(self, config: ModelPredictionConfig):
        self.config = config
        self.model = YOLO(self.config.trained_weights_path)
        # Assume default confidence threshold if not specified in params
        self.confidence_threshold = self.config.confidence_threshold

    def run_inference(self):
        # Ensure test_images_path directory exists
        assert os.path.isdir(self.config.test_images_path), f"Directory does not exist: {self.config.test_images_path}"
        
        # Collect image file paths
        image_paths = [os.path.join(self.config.test_images_path, image_file) for image_file in os.listdir(self.config.test_images_path) if image_file.endswith(('.jpg', '.png', '.jpeg'))]  # Ensure these are files
        
        # Ensure results_path directory exists or create it
        Path(self.config.results_path).mkdir(parents=True, exist_ok=True)
        
        results = self.model(image_paths, conf=self.confidence_threshold)  # Assuming this correctly handles a list of image file paths
        
        for idx, result in enumerate(results):
            original_filename = os.path.basename(image_paths[idx])
            result_filename = f"{original_filename.rsplit('.', 1)[0]}_result.jpg"
            result_path = os.path.join(self.config.results_path, result_filename)
            
            # Assuming result has a method to save to a file path
            result.save(result_path)
            print(f"Result saved to {result_path}")

if __name__ == "__main__":
    config_manager = ConfigurationManager()
    model_prediction_config = config_manager.get_model_prediction_config()

    # Make sure directories exist or are created
    create_directories([model_prediction_config.results_path])

    inference = YOLOInference(model_prediction_config)
    inference.run_inference()



[2024-02-28 00:09:14,640: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-02-28 00:09:14,649: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-28 00:09:14,650: INFO: common: created directory at: test/test_results]


FileNotFoundError: [Errno 2] No such file or directory: 'missile_trained_weights\\best.pt'

In [19]:
from flask import Flask, request, render_template, redirect, url_for, send_from_directory
import os
from pathlib import Path
# Assuming YOLOInference and ModelPredictionConfig are defined in your project as indicated
from YOLOv8_objectdetection.components.model_prediction import YOLOInference
from YOLOv8_objectdetection.entity.config_entity import ModelPredictionConfig

# Initialize Flask application
app = Flask(__name__)

# Configuration for YOLO model (adjust paths as necessary)
config = ModelPredictionConfig(
    trained_weights_path='website/best.pt',  # Adjust this path
    confidence_threshold=0.5,
    test_images_path='static/uploads',
    results_path='static/results'
)

# Ensure directories exist
os.makedirs(config.test_images_path, exist_ok=True)
os.makedirs(config.results_path, exist_ok=True)

# Configure upload folder in Flask app
app.config['UPLOAD_FOLDER'] = config.test_images_path
app.config['RESULTS_FOLDER'] = config.results_path

# Initialize YOLOInference with config
yolo_inference = YOLOInference(config)

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # Check if the post request has the file part
        if 'file' not in request.files:
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            # No file selected for uploading
            return redirect(request.url)
        if file:
            # Save the file to the specified upload folder
            filepath = os.path.join(app.config['UPLOAD_FOLDER'], file.filename)
            file.save(filepath)
            
            # Here you would call your inference method
            # Note: Adjust your YOLOInference class or its usage as needed to fit this workflow
            # For demonstration, the call is commented out
            # yolo_inference.run_inference()

            # Redirect to the results page or another appropriate action
            return redirect(url_for('results'))
    
    return render_template('upload.html')

@app.route('/results')
def results():
    # List results files (adjust as needed)
    results_files = os.listdir(app.config['RESULTS_FOLDER'])
    results_files = [os.path.join('/static/results', file) for file in results_files]
    return render_template('results.html', results_files=results_files)

@app.route('/static/results/<filename>')
def result(filename):
    return send_from_directory(app.config['RESULTS_FOLDER'], filename)

if __name__ == '__main__':
    app.run(debug=True)


[2024-02-28 00:15:12,874: INFO: model_prediction: YOLO model initialized with weights from website/best.pt and confidence threshold set to 0.5]
 * Serving Flask app '__main__'
 * Debug mode: on
[2024-02-28 00:15:12,911: INFO: _internal: WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000]
[2024-02-28 00:15:12,912: INFO: _internal: Press CTRL+C to quit]
[2024-02-28 00:15:12,918: INFO: _internal:  * Restarting with stat]


SystemExit: 1

c:\Users\vishw\anaconda3\envs\capstone\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [20]:
%pwd

'c:\\Users\\vishw\\Documents\\cap_project_tulasi\\YOLO-missile-detection-capstone-project'